In [12]:
import time
import sys
import os
import re
import pandas as pd
import numpy as np
import pyperclip  # 引入模块

In [13]:
def recover(x):
    if x=='TNF':
        return('THE NORTH FACE/北面')
    elif x=='CanadaGoose':
        return('Canada Goose')
    elif x=='NEW_BALANCE':
        return('NEW BALANCE')
    elif x=='TommyHilfiger':
        return('TOMMY HILFIGER')
    elif x=='Jackwolfskin/狼爪':
        return('Jack wolfskin/狼爪')
    elif x=='ARCTERYX/始祖鸟':
        return('ARC‘TERYX/始祖鸟')
    else:
        return(x)

In [14]:
def brand_N(changed):
    brand_name=[]
    if 'NEW BALANCE' in changed:
        changed=changed.replace('NEW BALANCE','NEW_BALANCE')
    if 'Canada Goose' in changed:
        changed=changed.replace('Canada Goose','CanadaGoose')
    if 'TOMMY HILFIGER' in changed:
        changed=changed.replace('TOMMY HILFIGER','TommyHilfiger')   
    if 'THE NORTH FACE/北面' in changed:
        changed=changed.replace('THE NORTH FACE/北面','TNF')    
    if 'Jack wolfskin/狼爪' in changed:
        changed=changed.replace('Jack wolfskin/狼爪','Jackwolfskin/狼爪')  
    if 'ARC‘TERYX/始祖鸟' in changed:
        changed=changed.replace('ARC‘TERYX/始祖鸟','ARCTERYX/始祖鸟')       
    brand1= changed.split('支付转化指数 ')
    brand3=[x for x in brand1[1].split(' ') if x!='']
    for b in brand3:
        if bool(re.search(r'\d', b))==False:
            brand_name.append(b.strip())
    return(brand_name)

In [15]:
def gender_build(changed):
    title_g=['女','男','未知 ']
    col_nums=len(title_g)
    gender_l2=[]
    gender_l1= changed.split(' ')
    for g in gender_l1:
        if bool(re.search(r'\d', g)):
            gender_l2.append(g)
    t=pd.DataFrame([gender_l2[i:i+col_nums] for i in range(0,len(gender_l2),col_nums)],columns=title_g)
    t2=t.drop(["未知 "],axis=1,inplace=False)
    t2.index=brand_name
    return(t2)

In [16]:
def age_build(changed):
    title_g=[]
    age_l2=[]
    age_l1= changed.split(' ')
    for g in age_l1:
        if (g.find('~')>=0) | (g.find('>') >=0):
            if g not in title_g:
                title_g.append(g)
        else:        
            if bool(re.search(r'\d', g)):
                age_l2.append(g)
            
    col_nums=len(title_g)        
    t=pd.DataFrame([age_l2[i:i+col_nums] for i in range(0,len(age_l2),col_nums)],columns=title_g)
    t.index=brand_name
    return(t)

In [17]:
def city_build(changed):
    top=dict()
    city_l1= changed.split('排名 省份 客群指数 ')
    city_l2=city_l1[1:]
    for num, i in enumerate(city_l2):
        top[num]=pd.Series([x for x in i.split(' ') if max(x.find("市"),x.find("特别行政区"))>-1 ])
    t=pd.concat(top,axis=1).reset_index(drop=True)
    t.columns=brand_name
    return(t)

In [18]:
def brand_build(changed):
    top=dict()
    gender_l1= changed.split('品牌名称 偏好人数 ')
    gender_l2=gender_l1[1:]
    gender_l2=[x.replace('Canada Goose','CanadaGoose') for x in gender_l2]
    gender_l2=[x.replace('THE NORTH FACE/北面','TNF') for x in gender_l2]
    gender_l2=[x.replace('NEW BALANCE','NEW_BALANCE') for x in gender_l2]
    gender_l2=[x.replace('361°','三六一度') for x in gender_l2]
    gender_l2=[x.replace('NEW BOLUNE/新百伦','NEW_BOLUNE/新百伦') for x in gender_l2]
    for num, i in enumerate(gender_l2):
        top[num]=pd.Series([x for x in i.split(' ') if (bool(re.search(r'\d', x))==False) & (x!='') ])
    t=pd.concat(top,axis=1).reset_index(drop=True)
    t.columns=brand_name
    t2=t.applymap(recover)
    return(t2)

In [19]:
def cat_build(changed):
    top=dict()
    gender_l1= changed.split('类目名称 偏好人数 ')
    gender_l2=gender_l1[1:]
    for num, i in enumerate(gender_l2):
        top[num]=pd.Series([x for x in i.split(' ') if (bool(re.search(r'\d', x))==False) & (x!='') ])
    t=pd.concat(top,axis=1).reset_index(drop=True)
    return(t)

In [20]:
def pay_build(changed):
    pay_l1= changed.split('支付金额 客群占比 ')
    title_g=[]
    pay_l2=[]
    for g1 in pay_l1:
        g2=g1.split(' ')
        for g in g2:
            if (g.find('-')>=0) | (g.find('以上')>=0):
                if g not in title_g:
                    title_g.append(g)
            else:        
                if (g.find('%')>=0):
                    pay_l2.append(g)
                    
    col_nums=len(title_g)        
    t=pd.DataFrame([pay_l2[i:i+col_nums] for i in range(0,len(pay_l2),col_nums)],columns=title_g)
    t2=t.T.reset_index()
    return(t2)

In [21]:
recent_value = ""
tmp_value="" # 初始化（应该也可以没有这一行，感觉意义不大。但是对recent_value的初始化是必须的）

In [22]:
pyperclip.copy('')

city_d=dict()
while True:
    tmp_value = pyperclip.paste() 			# 读取剪切板复制的内容
    
    try:
        if tmp_value != recent_value:				 #如果检测到剪切板内容有改动，那么就进入文本的修改
            recent_value = tmp_value
            changed = re.sub(r"\s{2,}", " ", recent_value) 	#将文本的换行符去掉，变成一个空格
            pyperclip.copy(changed) 							#将修改后的文本写入系统剪切板中
            if '交易指数' in changed:
                brand_name=brand_N(changed)
            if '性别' in changed:
                gender=gender_build(changed)
                
            if '年龄' in changed:
                age=age_build(changed)
                
            if '城市' in changed:
                if '1 ／3' in changed:
                    city_d[0]= city_build(changed)
                if '2 ／3' in changed:
                    city_d[1]= city_build(changed)
                if '3 ／3' in changed:
                    city_d[2]= city_build(changed)
                
                city=pd.concat(city_d,axis=0).reset_index(drop=True)


            if '品牌' in changed:
                brand= brand_build(changed)
     
            if '类目' in changed:
                cat= cat_build(changed)
        
            if '支付' in changed:
                if '交易指数' not in changed:
                    pay= pay_build(changed)
           
        time.sleep(0.1)
    except KeyboardInterrupt:  # 如果有ctrl+c，那么就退出这个程序。  （不过好像并没有用。无伤大雅）
        break
        
    if tmp_value == 'getend': # 如果复制的是getend，就退出程序。（这个主要是为了方便我在spyder中运行、退出的时候用的。）
        break

In [661]:
#接下来就是复制生意参谋的数

#复制: getend
#复制后上面的程序就会中止，每一次更新页面都需要中止重新做一次 

In [662]:
#后续的每跑一个格子，粘贴1次到excel即可，跳过ctrl+c ，直接按ctrl+v

In [23]:
print(brand_name)

['Kipling', 'LeSportsac', 'Herschel']


In [24]:
#性别分析
gender.to_clipboard(excel=True,index=False,header=False)
print(gender)

                 女      男
Kipling     13,556  4,343
LeSportsac   8,036  1,943
Herschel     4,515  4,849


In [25]:
#年龄分析
age.to_clipboard(excel=True,index=False,header=False)
print(age)

            18~24  25~29  30~34  35~39  40~49   >=50
Kipling     3,059  4,670  6,482  5,945  5,847  4,191
LeSportsac  1,002  2,152  4,001  4,123  3,518  1,796
Herschel    4,346  3,223  2,210  1,378  1,420    727


In [26]:
#top城市
city.index=range(1,len(city)+1)
city.columns=brand_name
print(city)
city.to_clipboard(excel=True,index=False,header=False)

   Kipling LeSportsac Herschel
1      北京市        北京市      上海市
2      上海市        上海市      北京市
3      广州市        南京市      广州市
4      深圳市        广州市      杭州市
5      西安市        成都市      深圳市
6      南京市        深圳市      南京市
7      成都市        杭州市      成都市
8      杭州市        武汉市      武汉市
9      武汉市        西安市      重庆市
10     郑州市        郑州市      长沙市
11     天津市        重庆市      西安市
12     沈阳市        天津市  香港特别行政区
13     济南市        昆明市      东莞市
14     重庆市        沈阳市      天津市
15     长沙市        长沙市      苏州市
16     大连市       哈尔滨市      厦门市
17    石家庄市        长春市  澳门特别行政区
18    哈尔滨市        济南市      福州市
19     长春市       石家庄市      合肥市
20     苏州市        大连市      济南市
21     合肥市        苏州市      昆明市
22     太原市        合肥市      沈阳市
23     昆明市        青岛市      郑州市
24     厦门市        福州市      长春市
25     东莞市        无锡市      佛山市
26     福州市        宁波市      南宁市
27     青岛市    香港特别行政区      贵阳市
28     南宁市        厦门市     哈尔滨市
29     南昌市        太原市      海口市
30     贵阳市        南宁市     石家庄市


In [667]:
#品牌
brand.index=range(1,len(brand)+1)
brand.columns=brand_name
print(brand)
brand.to_clipboard(excel=True,index=False,header=False)

       TIMBERLAND/添柏岚  TommyHilfiger
1      TIMBERLAND/添柏岚          TOMMY
2   THE NORTH FACE/北面       HILFIGER
3         Adidas/阿迪达斯   TOMMYHFILGER
4          Uniqlo/优衣库      Lining/李宁
5             Nike/耐克   Tagkita/她及其他
6           Lining/李宁       COACH/蔻驰
7                 南极人         Calvin
8         Bananain/蕉内     Klein/凯文克莱
9             MIUI/小米            南极人
10         Levi’s/李维斯          Tommy
11                NaN       KIRKLAND
12                NaN  Signature/科克兰
13                NaN   Shiseido/资生堂


In [668]:
cat

,0,1
0,户外服装,户外服装
1,户外鞋靴,T恤
2,T恤,连衣裙
3,休闲裤,女士包袋
4,夹克,T恤
5,衬衫,裤子
6,T恤,短外套
7,手机配件,衬衫
8,连衣裙,毛针织衫
9,运动休闲鞋,低帮鞋


In [669]:
#类目
cat.index=range(1,len(cat)+1)
cat.columns=brand_name
print(cat)
cat.to_clipboard(excel=True,index=False,header=False)

   TIMBERLAND/添柏岚 TommyHilfiger
1            户外服装          户外服装
2            户外鞋靴            T恤
3              T恤           连衣裙
4             休闲裤          女士包袋
5              夹克            T恤
6              衬衫            裤子
7              T恤           短外套
8            手机配件            衬衫
9             连衣裙          毛针织衫
10          运动休闲鞋           低帮鞋


In [670]:
#支付
pay.columns=['spend_cut']+brand_name
pay.to_clipboard(excel=True,index=False,header=False)
print(pay)

     spend_cut TIMBERLAND/添柏岚 TommyHilfiger
0       0-20.0          5.11%         2.70%
1    20.0-45.0          5.68%         2.70%
2    45.0-70.0          1.70%        37.84%
3   70.0-120.0          3.41%        56.76%
4  120.0-275.0         15.34%          None
5      275.0以上         68.75%          None
